In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns

In [ ]:
# This is to read the visitation dataset into google colab

vd = pd.read_csv("sample_data/Visitation Data-Table 1.csv")
vd

,Year,Week,Mt. Baw Baw,Mt. Stirling,Mt. Hotham,Falls Creek,Mt. Buller,Selwyn,Thredbo,Perisher,Charlotte Pass
0,2014,1,555,60,3483,2790,8296,1041,5535,7370,408
1,2014,2,804,42,1253,1425,1987,383,2090,2751,151
2,2014,3,993,30,2992,2101,2413,597,3216,4255,230
3,2014,4,2976,165,9680,9544,18831,2877,15497,20265,1134
4,2014,5,11112,645,29628,26211,49217,8588,46546,61339,3403
...,...,...,...,...,...,...,...,...,...,...,...
160,2024,11,5977,1416,24260,38541,43387,6859,37608,49136,2667
161,2024,12,3597,701,19289,42101,36668,5969,33208,43382,2392
162,2024,13,1500,213,14487,25252,15415,3351,18120,23908,1323
163,2024,14,0,0,2700,17137,0,1177,6270,8244,454


Before reading the climate excel file, some data cleaning was done in Excel. This included removing the years 2010 to 2013. This also included removing all months and days except for the ski season (from 9th of June to 21st September).

For some reason, I kept 2025 in the years, which I should have got ridden of, but will not impact the final csv file used to create graphs and models.



In [ ]:
# This is to read the climate excel file into google colab

cd = pd.read_csv("sample_data/climatedata2025.csv")
cd

,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
0,71075,2014,6,9,3.4,-1.8,6.6
1,71075,2014,6,10,8.1,-5.0,3.8
2,71075,2014,6,11,13.4,-5.1,0.0
3,71075,2014,6,12,8.2,-0.6,0.0
4,71075,2014,6,13,4.7,2.1,0.0
...,...,...,...,...,...,...,...
8427,72161,2025,7,24,3.0,-2.7,4.2
8428,72161,2025,7,25,6.0,-2.4,0.2
8429,72161,2025,7,26,2.7,0.0,10.2
8430,72161,2025,7,27,2.5,1.3,15.6


In [ ]:
vd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Year            165 non-null    int64
 1   Week            165 non-null    int64
 2   Mt. Baw Baw     165 non-null    int64
 3   Mt. Stirling    165 non-null    int64
 4   Mt. Hotham      165 non-null    int64
 5   Falls Creek     165 non-null    int64
 6   Mt. Buller      165 non-null    int64
 7   Selwyn          165 non-null    int64
 8   Thredbo         165 non-null    int64
 9   Perisher        165 non-null    int64
 10  Charlotte Pass  165 non-null    int64
dtypes: int64(11)
memory usage: 14.3 KB


The following process for data cleaning will focus on visitation data

In [ ]:
visits_missing_values_count = vd.isnull().sum()
visits_missing_values_count

,0
Year,0
Week,0
Mt. Baw Baw,0
Mt. Stirling,0
Mt. Hotham,0
Falls Creek,0
Mt. Buller,0
Selwyn,0
Thredbo,0
Perisher,0


The above result shows that each category has no missing values. Thus, this dataset is fine.

In [ ]:
cd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8432 entries, 0 to 8431
Data columns (total 7 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Bureau of Meteorology station number  8432 non-null   int64  
 1   Year                                  8432 non-null   int64  
 2   Month                                 8432 non-null   int64  
 3   Day                                   8432 non-null   int64  
 4   Maximum temperature (Degree C)        8116 non-null   float64
 5   Minimum temperature (Degree C)        8125 non-null   float64
 6   Rainfall amount (millimetres)         8087 non-null   float64
dtypes: float64(3), int64(4)
memory usage: 461.3 KB


In [ ]:
climate_missing_values_count = cd.isnull().sum()
climate_missing_values_count

,0
Bureau of Meteorology station number,0
Year,0
Month,0
Day,0
Maximum temperature (Degree C),316
Minimum temperature (Degree C),307
Rainfall amount (millimetres),345


The climate dataset has missing values. In fact:
- Max temperature has 316 missing values
- Min temperature has 307 missing values
- Rainfall has 345 missing values

In [ ]:
# how many total missing values do we have?
total_cells = np.prod(cd.shape)
total_missing = climate_missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

1.6400108430468963


The total percentage of missing data is 1.64%.

Start of data cleaning process:

In [ ]:
# Forward filled na values for max and minimum temperatures for each ski resort respectively.

cleaned_cd = cd.copy()
cleaned_cd[['Maximum temperature (Degree C)', 'Minimum temperature (Degree C)']] = cleaned_cd[['Maximum temperature (Degree C)', 'Minimum temperature (Degree C)']].fillna(method="ffill")
cleaned_cd

/tmp/ipython-input-3334000867.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cleaned_cd[['Maximum temperature (Degree C)', 'Minimum temperature (Degree C)']] = cleaned_cd[['Maximum temperature (Degree C)', 'Minimum temperature (Degree C)']].fillna(method="ffill")


,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
0,71075,2014,6,9,3.4,-1.8,6.6
1,71075,2014,6,10,8.1,-5.0,3.8
2,71075,2014,6,11,13.4,-5.1,0.0
3,71075,2014,6,12,8.2,-0.6,0.0
4,71075,2014,6,13,4.7,2.1,0.0
...,...,...,...,...,...,...,...
8427,72161,2025,7,24,3.0,-2.7,4.2
8428,72161,2025,7,25,6.0,-2.4,0.2
8429,72161,2025,7,26,2.7,0.0,10.2
8430,72161,2025,7,27,2.5,1.3,15.6


In [ ]:
# Filled na values in rainfall amount with 0, as there is a good chance that there was no rainfall that day.

cleaned_cd[['Rainfall amount (millimetres)']] = cleaned_cd[['Rainfall amount (millimetres)']].fillna(0)
cleaned_cd

,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
0,71075,2014,6,9,3.4,-1.8,6.6
1,71075,2014,6,10,8.1,-5.0,3.8
2,71075,2014,6,11,13.4,-5.1,0.0
3,71075,2014,6,12,8.2,-0.6,0.0
4,71075,2014,6,13,4.7,2.1,0.0
...,...,...,...,...,...,...,...
8427,72161,2025,7,24,3.0,-2.7,4.2
8428,72161,2025,7,25,6.0,-2.4,0.2
8429,72161,2025,7,26,2.7,0.0,10.2
8430,72161,2025,7,27,2.5,1.3,15.6


In [ ]:
# Checking if there are missing values (none, which is good!)

cleaned_cd_missing_values = cleaned_cd.isnull().sum()
cleaned_cd_missing_values

,0
Bureau of Meteorology station number,0
Year,0
Month,0
Day,0
Maximum temperature (Degree C),0
Minimum temperature (Degree C),0
Rainfall amount (millimetres),0


In [ ]:
# Creating a date column

cleaned_cd['Date'] = pd.to_datetime(cd[['Year', 'Month', 'Day']])
cleaned_cd

,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres),Date
0,71075,2014,6,9,3.4,-1.8,6.6,2014-06-09
1,71075,2014,6,10,8.1,-5.0,3.8,2014-06-10
2,71075,2014,6,11,13.4,-5.1,0.0,2014-06-11
3,71075,2014,6,12,8.2,-0.6,0.0,2014-06-12
4,71075,2014,6,13,4.7,2.1,0.0,2014-06-13
...,...,...,...,...,...,...,...,...
8427,72161,2025,7,24,3.0,-2.7,4.2,2025-07-24
8428,72161,2025,7,25,6.0,-2.4,0.2,2025-07-25
8429,72161,2025,7,26,2.7,0.0,10.2,2025-07-26
8430,72161,2025,7,27,2.5,1.3,15.6,2025-07-27


In [ ]:
# Calculating weekly averages for each measurement

main_climate_data = cleaned_cd.copy()


# Step 1: Sort by Station Number and Date
main_climate_data = main_climate_data.sort_values(['Bureau of Meteorology station number', 'Year', 'Date']).reset_index(drop=True)

# Step 2: Assign a weekly group within each Station
main_climate_data['Week'] = main_climate_data.groupby(['Bureau of Meteorology station number', 'Year']).cumcount() // 7 + 1

# Step 3: Group by Station and Week, then average
weekly_summary = main_climate_data.groupby(['Bureau of Meteorology station number', 'Year', 'Week'])[
    ['Maximum temperature (Degree C)', 'Minimum temperature (Degree C)', 'Rainfall amount (millimetres)']
].mean().reset_index()

# Step 4: Optional – rename week label
weekly_summary['Week'] = 'Week ' + weekly_summary['Week'].astype(str)




This following code is just to export the new modifications (deleting, imputing etc.) into a csv file

In [ ]:
weekly_summary.to_csv('station_weekly_summary.csv', index=False)
from google.colab import files
files.download('station_weekly_summary.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
weekly_summary

,Bureau of Meteorology station number,Year,Week,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
0,71032,2014,Week 1,4.714286,-2.000000,5.800000
1,71032,2014,Week 2,2.657143,-1.685714,1.142857
2,71032,2014,Week 3,-1.028571,-4.114286,0.600000
3,71032,2014,Week 4,-1.314286,-4.357143,0.057143
4,71032,2014,Week 5,-1.685714,-5.185714,0.428571
...,...,...,...,...,...,...
1205,85291,2025,Week 4,4.328571,-0.485714,18.800000
1206,85291,2025,Week 5,2.014286,-1.457143,9.028571
1207,85291,2025,Week 6,1.542857,-2.214286,2.828571
1208,85291,2025,Week 7,2.328571,-1.457143,4.771429


There are many rows compared to the visits excel file. However, the climate file combines all ski resorts into one long column. In addition, I will not be using 2025 data. Thus, this size is actually to be expected.

In this next step, I went back to Excel to format the tables and combined them into 1 csv file. This involved matching the Bureau of Meteorology station number with ski resorts. Some assumptions have been made due to incomplete data for some ski resorts:

- The following ski resorts (Thredbo, Perisher, Mt. Buller, Falls Creek, Mt.Hotham and Mt. Baw Baw) each have their respective meteorology station to connect. Thus, these are fine.

- Mt. Stirling is close to Mt. Buller, so the measurements for Mt. Stirling are the same as Mt. Buller

- Selwyn ski resort is close to Cabramurra, so the Cabramurra measurements are used for Selwyn

- Charlotte Pass is nearly equidistant from Thredbo and Perisher. The measurements for Charlotte Pass were calculated as an average between Thredbo and Perisher. This can be seen in the excel formulas in the final csv file.



In [ ]:
dt = pd.read_csv("Full Visits Climate Data.csv")
dt

,Year/Week,Year,Week,Mt. Baw Baw Visits,Mt. Baw Baw Maximum temperature (Degree C),Mt. Baw Baw Minimum temperature (Degree C),Mt. Baw Baw Rainfall amount (millimetres),Mt. Stirling Visits,Mt. Stirling Maximum temperature (Degree C),Mt. Stirling Minimum temperature (Degree C),...,Perisher Minimum temperature (Degree C),Perisher Rainfall amount (millimetres),Charlotte Pass Visits,Charlotte Maximum temperature (Degree C),Charlotte Minimum temperature (Degree C),Charlotte Rainfall amount (millimetres),Average Pass Visits,Average Maximum temperature (Degree C),Average Minimum temperature (Degree C),Average Rainfall amount (millimetres)
0,2014/1,2014,1,555,4.957143,1.100000,3.485714,60,4.271429,-0.014286,...,-1.585714,8.857143,408,5.521429,-1.792857,7.328571,3282.000000,4.994444,-0.464286,4.788889
1,2014/2,2014,2,804,5.485714,0.814286,3.000000,42,5.257143,-0.342857,...,-1.857143,1.971429,151,3.957143,-1.771429,1.557143,1209.555556,4.666667,-0.577778,2.919048
2,2014/3,2014,3,993,1.042857,-1.814286,10.914286,30,0.385714,-2.514286,...,-2.457143,31.514286,230,-0.071429,-3.285714,16.057143,1869.666667,0.274603,-2.630159,13.971429
3,2014/4,2014,4,2976,2.528571,-0.957143,11.714286,165,0.957143,-1.857143,...,-2.457143,7.685714,1134,0.814286,-3.407143,3.871429,8996.555556,1.158730,-2.423016,6.480952
4,2014/5,2014,5,11112,0.685714,-2.257143,11.800000,645,-0.285714,-3.400000,...,-3.757143,5.571429,3403,-0.250000,-4.471429,3.000000,26298.777780,-0.215079,-3.609524,5.546032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,2024/11,2024,11,5977,6.157143,1.385714,3.742857,1416,4.885714,0.828571,...,0.085714,4.200000,2667,4.900000,-0.307143,4.228571,23316.777780,5.169841,0.559524,4.946032
161,2024/12,2024,12,3597,5.428571,0.928571,3.371429,701,4.257143,0.157143,...,0.671429,13.257143,2392,3.542857,-0.250000,11.585714,20811.888890,4.211111,0.183333,8.877778
162,2024/13,2024,13,1500,7.142857,-0.100000,7.228571,213,6.100000,-0.671429,...,0.671429,5.857143,1323,6.300000,-0.014286,4.614286,11507.666670,6.411111,-0.141270,5.252381
163,2024/14,2024,14,0,5.628571,0.528571,6.542857,0,5.728571,-0.571429,...,-0.728571,0.771429,454,7.414286,-0.985714,1.200000,3998.000000,6.509524,-0.334921,3.171429
